In [1]:
import numpy as np
import pandas as pd


In [2]:
def table_gen(cvs):
    #give cvs file, returns df
    season_table = {}
    # d = {'col1': [1, 2], 'col2': [3, 4]}
    for index, row in cvs.iterrows():
        # print(str(row['Team 1']), str(row['FT']), str(row['Team 2']))
        home = row['Team 1'][:-4].strip()
        away = row['Team 2'][:-4].strip()
        homeround = int(row['Team 1'][-4:].replace('(', '').replace(')',''))
        awayround = int(row['Team 2'][-4:].replace('(', '').replace(')',''))
        string_score = row['FT'].split('-')
        # cast as int
        homescore = int(string_score[0])
        awayscore = int(string_score[1])

        if home not in season_table:
            #####################[HWin, HLoss, HDraw, HGF, HGA,  HGD   HPoints, AWin, ALoss, ADraw, AGF, AGA, AGD APoints]
            ######################[0,    1,     2,     3,   4,    5,      6,     7,    8,     9,    10,   11,  12,  13]
            season_table[home] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        if away not in season_table:
            season_table[away] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        #update goals for & against
        season_table[home][3] = season_table[home][3] + homescore #gf    
        season_table[home][4] = season_table[home][4] + awayscore #ga
        # update GD
        season_table[home][5] = season_table[home][3] - season_table[home][4]# - season_table[home][4]) 

        season_table[away][10] = season_table[away][10] + awayscore #gf
        season_table[away][11] = season_table[away][11] + homescore #ga
        # update GD
        season_table[away][12] = season_table[away][10] - season_table[away][11]

        #####################[HWin, HLoss, HDraw, HGF, HGA,  HGD   HPoints, AWin, ALoss, ADraw, AGF, AGA, AGD APoints]
        ######################[0,    1,     2,     3,   4,    5,      6,     7,    8,     9,    10,   11,  12,  13]
        if homescore > awayscore:
            season_table[home][6] += 3 #points for win
            season_table[home][0] += 1 #add win
            season_table[away][8] += 1 #add loss
        elif homescore < awayscore:
            season_table[away][13] += 3 #points for win
            season_table[away][7] += 1 #add win
            season_table[home][1] += 1 #add loss
        else:
            #draw
            season_table[home][6] += 1 #points for draw
            season_table[away][13] += 1 #points for draw
            season_table[away][2] += 1 #add draw
            season_table[home][9] += 1 #add draw
    # done
    df = pd.DataFrame(data=season_table)
    df = df.T    
    df.columns = ['Home Win', 'Home Loss', 'Home Draw', 'Home GF', 'Home GA', 'Home GD', 'Home Points', 
              'Away Win', 'Away Loss', 'Away Draw', 'Away GF', 'Away GA', 'Away GD', 'Away Points']
    df['Win'] = df.apply (lambda row: row['Home Win'] + row['Away Win'], axis=1)
    df['Loss'] = df.apply (lambda row: row['Home Loss'] + row['Away Loss'], axis=1)
    df['Draw'] = df.apply (lambda row: row['Home Draw'] + row['Away Draw'], axis=1)
    df['GF'] = df.apply (lambda row: row['Home GF'] + row['Away GF'], axis=1)
    df['GA'] = df.apply (lambda row: row['Home GA'] + row['Away GA'], axis=1)
    df['GD'] = df.apply (lambda row: row['Home GD'] + row['Away GD'], axis=1)
    df['Points'] = df.apply (lambda row: row['Home Points'] + row['Away Points'], axis=1)
    df.sort_values(['Points', 'GD'], ascending=False, inplace=True)
    
    values = list(range(1, 21))
    data = np.array(values)
    s = pd.Series(data)
    df['Rank'] = s.values
    df.set_index('Rank', inplace=True)
    return df
        

        
    


In [13]:
#season_2010 = pd.read_csv('eng-england/2010s/2010-11/1-premierleague.csv')
#season_2011 = pd.read_csv('eng-england/2010s/2011-12/1-premierleague.csv')
#season_2012 = pd.read_csv('eng-england/2010s/2012-13/1-premierleague.csv')
#eason_2013 = pd.read_csv('eng-england/2010s/2013-14/1-premierleague.csv')
#season_2014 = pd.read_csv('eng-england/2010s/2014-15/1-premierleague.csv')
#season_2015 = pd.read_csv('eng-england/2010s/2015-16/1-premierleague.csv')
#season_2016 = pd.read_csv('eng-england/2010s/2016-17/1-premierleague.csv')
#season_2017 = pd.read_csv('eng-england/2010s/2017-18/1-premierleague.csv')
dir_contents = ['1995-96', '1996-97', '1997-98', '1998-99', '1999-00', '2000-01',
                '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07',
                '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13',
                '2013-14', '2014-15', '2015-16', '2016-17', '2017-18']
season_series = []


for year in dir_contents:
    filename = 'league_results/' + year + '.csv'
    season = pd.read_csv(filename)
    season_table = table_gen(season)
    season_table
    season_series.append(season_table)

# t_2017 = table_gen(season_2017)
season_series_obj = pd.Series(season_series)
df_concat = pd.concat(season_series)
by_row_index = df_concat.groupby(df_concat.index)
df_means = by_row_index.mean()
df_max = by_row_index.max()
df_min = by_row_index.min()


In [12]:
df_means

,Home Win,Home Loss,Home Draw,Home GF,Home GA,Home GD,Home Points,Away Win,Away Loss,Away Draw,...,Away GA,Away GD,Away Points,Win,Loss,Draw,GF,GA,GD,Points
Rank,,,,,,,,,,,,,,,,,,,,,
1,15.434783,1.043478,5.000000,46.913043,13.391304,33.521739,48.826087,10.869565,3.130435,2.521739,...,18.652174,15.652174,37.608696,26.304348,4.173913,7.521739,81.217391,32.043478,49.173913,86.434783
2,13.913043,1.521739,4.695652,42.217391,13.217391,29.000000,45.304348,9.782609,4.521739,3.565217,...,19.652174,12.217391,34.043478,23.695652,6.043478,8.260870,74.086957,32.869565,41.217391,79.347826
3,12.652174,2.217391,4.608696,37.608696,14.086957,23.521739,42.086957,9.130435,5.260870,4.130435,...,21.391304,9.304348,32.000000,21.782609,7.478261,8.739130,68.304348,35.478261,32.826087,74.086957
4,11.956522,2.956522,5.434783,38.043478,15.608696,22.434783,39.956522,7.826087,5.739130,4.086957,...,22.956522,4.652174,28.913043,19.782609,8.695652,9.521739,65.652174,38.565217,27.086957,68.869565
5,10.913043,3.217391,4.869565,31.956522,17.478261,14.478261,37.608696,7.304348,6.826087,4.869565,...,24.217391,0.521739,26.782609,18.217391,10.043478,9.739130,56.695652,41.695652,15.000000,64.391304
6,10.391304,3.565217,5.434783,33.391304,19.347826,14.043478,36.217391,6.521739,7.043478,5.043478,...,24.652174,-1.000000,25.000000,16.913043,10.608696,10.478261,57.043478,44.000000,13.043478,61.217391
7,9.913043,4.043478,5.173913,30.869565,18.652174,12.217391,34.782609,5.869565,7.956522,5.043478,...,26.739130,-4.217391,22.782609,15.782609,12.000000,10.217391,53.391304,45.391304,8.000000,57.565217
8,8.782609,4.260870,4.956522,27.391304,19.173913,8.217391,32.304348,5.521739,8.521739,5.956522,...,28.000000,-6.217391,21.521739,14.304348,12.782609,10.913043,49.173913,47.173913,2.000000,53.826087
9,9.217391,4.826087,5.217391,28.521739,21.478261,7.043478,32.608696,4.521739,9.260870,4.956522,...,30.565217,-11.130435,18.782609,13.739130,14.086957,10.173913,47.956522,52.043478,-4.086957,51.391304


In [14]:
df_max

,Home Win,Home Loss,Home Draw,Home GF,Home GA,Home GD,Home Points,Away Win,Away Loss,Away Draw,...,Away GA,Away GD,Away Points,Win,Loss,Draw,GF,GA,GD,Points
Rank,,,,,,,,,,,,,,,,,,,,,
1,18,3,10,68,25,54,55,16,6,6,...,29,32,50,32,6,13,106,45,79,100
2,17,3,8,54,20,40,53,13,7,7,...,32,23,43,28,9,12,101,50,60,89
3,15,6,9,48,18,33,48,14,9,7,...,32,23,44,25,11,15,87,49,44,83
4,14,5,8,48,23,35,45,11,11,7,...,31,16,35,24,15,13,84,52,46,79
5,14,6,12,41,26,23,45,10,11,9,...,34,10,34,23,14,17,77,54,33,75
6,15,6,9,54,28,34,47,10,11,10,...,33,11,35,21,14,15,74,55,28,69
7,14,7,8,44,29,28,44,10,12,10,...,43,13,34,19,15,15,71,57,28,64
8,13,7,9,40,25,22,43,9,12,9,...,39,5,29,16,17,16,63,58,13,61
9,12,8,7,36,29,15,39,7,12,9,...,40,-2,25,16,17,15,56,67,6,58
